# Animal Shelter Outcomes
---

Problem Statement...

### Background
...

---
## Data Cleaning

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
# reading in shelter intakes
intakes =  pd.read_csv('../data/austin_animal_center_intakes_20241017.csv')
intakes.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A857105,Johnny Ringo,05/12/2022 12:23:00 AM,May 2022,4404 Sarasota Drive in Austin (TX),Public Assist,Normal,Cat,Neutered Male,2 years,Domestic Shorthair,Orange Tabby


In [3]:
intakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168040 entries, 0 to 168039
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         168040 non-null  object
 1   Name              119647 non-null  object
 2   DateTime          168040 non-null  object
 3   MonthYear         168040 non-null  object
 4   Found Location    168040 non-null  object
 5   Intake Type       168040 non-null  object
 6   Intake Condition  168040 non-null  object
 7   Animal Type       168040 non-null  object
 8   Sex upon Intake   168038 non-null  object
 9   Age upon Intake   168039 non-null  object
 10  Breed             168040 non-null  object
 11  Color             168040 non-null  object
dtypes: object(12)
memory usage: 15.4+ MB


*The Name column has a lot of null values and we're assuming a pet's name won't affect their chances of adoption, so going to drop this column. Also dropping MonthYear because that information is also in the DateTime column.*

In [4]:
# dropping inconsequential columns
intakes.drop(columns=['Name', 'MonthYear'], inplace=True)

In [5]:
# renaming columns to be intake specific and snake case
columns = {
    'Animal ID': 'animal_id',
    'DateTime': 'intake_time',
    'Found Location': 'found_location',
    'Intake Type': 'intake_type',
    'Intake Condition': 'intake_condition',
    'Animal Type': 'animal_type',
    'Sex upon Intake': 'intake_gender',
    'Age upon Intake': 'intake_age',
    'Breed': 'intake_breed',
    'Color': 'intake_color'    
}

intakes = intakes.rename(columns=columns)

In [6]:
# converting intake_time column to datetime format
intakes['intake_time'] = pd.to_datetime(intakes['intake_time'], format='%m/%d/%Y %H:%M:%S %p')

In [7]:
intakes.nunique()

animal_id           151007
intake_time         115671
found_location       68164
intake_type              6
intake_condition        20
animal_type              5
intake_gender            5
intake_age              55
intake_breed          2969
intake_color           651
dtype: int64

In [8]:
intakes.sort_values(by=['animal_id','intake_time'], inplace=True)
intakes.head()

,animal_id,intake_time,found_location,intake_type,intake_condition,animal_type,intake_gender,intake_age,intake_breed,intake_color
113926,A006100,2014-03-07 02:26:00,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
5413,A006100,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
25241,A006100,2017-12-07 02:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
88544,A047759,2014-04-02 03:55:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,10 years,Dachshund,Tricolor
120450,A134067,2013-11-16 09:02:00,12034 Research Blvd in Austin (TX),Public Assist,Injured,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White


*There appear to be some animals that were in a shelter multiple times, going to create column with a unique animal_id for each stay before merging with outcomes data.*

In [9]:
intakes['stay'] = intakes.groupby('animal_id').cumcount() +1 

In [10]:
intakes.nunique()

animal_id           151007
intake_time         115671
found_location       68164
intake_type              6
intake_condition        20
animal_type              5
intake_gender            5
intake_age              55
intake_breed          2969
intake_color           651
stay                    33
dtype: int64

In [11]:
intakes['animal_stay'] = intakes['animal_id'] + '-' + intakes['stay'].astype('str')

In [12]:
# dropping animal_id and stay columns now they're incorporated in animal_stay
intakes.drop(columns=['animal_id', 'stay'], inplace=True)

In [13]:
# reading in shelter outcomes
outcomes = pd.read_csv('../data/austin_animal_center_outcomes_20241017.csv')
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A882831,*Hamilton,07/01/2023 06:12:00 PM,Jul 2023,03/25/2023,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White
1,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
2,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
3,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [14]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167942 entries, 0 to 167941
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         167942 non-null  object
 1   Name              119733 non-null  object
 2   DateTime          167942 non-null  object
 3   MonthYear         167942 non-null  object
 4   Date of Birth     167942 non-null  object
 5   Outcome Type      167896 non-null  object
 6   Outcome Subtype   77144 non-null   object
 7   Animal Type       167942 non-null  object
 8   Sex upon Outcome  167940 non-null  object
 9   Age upon Outcome  167926 non-null  object
 10  Breed             167942 non-null  object
 11  Color             167942 non-null  object
dtypes: object(12)
memory usage: 15.4+ MB


*Dropping Name and MonthYear columns for outcomes data as well as Outcome Subtype. This column has even more null values than Name and we'll be focusing on the primary Outcome Type only.*

In [15]:
# dropping inconsequential columns
outcomes.drop(columns=['Name', 'MonthYear', 'Outcome Subtype'], inplace=True)

In [16]:
# renaming columns to be outcome specific and snake case
outcome_columns = {
    'Animal ID': 'animal_id',
    'DateTime': 'outcome_time',
    'Date of Birth': 'date_of_birth',
    'Outcome Type': 'outcome_type',
    'Animal Type': 'outcome_animal_type',
    'Sex upon Outcome': 'outcome_gender',
    'Age upon Outcome': 'outcome_age',
    'Breed': 'outcome_breed',
    'Color': 'outcome_color'    
}

outcomes = outcomes.rename(columns=outcome_columns)

In [17]:
# converting outcome_time and date of birth columns to datetime format
outcomes['outcome_time'] = pd.to_datetime(outcomes['outcome_time'], format='%m/%d/%Y %H:%M:%S %p')
outcomes['date_of_birth'] = pd.to_datetime(outcomes['date_of_birth'], format='%m/%d/%Y')

In [18]:
outcomes.nunique()

animal_id              150912
outcome_time           139971
date_of_birth            8501
outcome_type               11
outcome_animal_type         5
outcome_gender              5
outcome_age                55
outcome_breed            2969
outcome_color             653
dtype: int64

In [19]:
outcomes.sort_values(by=['animal_id','outcome_time'], inplace=True)
outcomes.head()

,animal_id,outcome_time,date_of_birth,outcome_type,outcome_animal_type,outcome_gender,outcome_age,outcome_breed,outcome_color
145647,A006100,2014-03-08 05:10:00,2007-07-09,Return to Owner,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
71767,A006100,2014-12-20 04:35:00,2007-07-09,Return to Owner,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
128276,A006100,2017-12-07 12:00:00,2007-07-09,Return to Owner,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
49549,A047759,2014-04-07 03:12:00,2004-04-02,Transfer,Dog,Neutered Male,10 years,Dachshund,Tricolor
102988,A134067,2013-11-16 11:54:00,1997-10-16,Return to Owner,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White


*Need to create stay specific column for outcomes as well.*

In [20]:
outcomes['stay'] = outcomes.groupby('animal_id').cumcount() +1 

In [21]:
outcomes['animal_stay'] = outcomes['animal_id'] + '-' + (outcomes['stay']).astype('str')

In [22]:
# dropping animal_id and stay colums for outcomes
outcomes.drop(columns=['animal_id', 'stay'], inplace=True)

## Merge DataFrames

In [23]:
# meging intakes and outcomes
intakes_outcomes = pd.merge(left=intakes, right=outcomes, how='inner', on='animal_stay')

In [24]:
intakes_outcomes.head()

,intake_time,found_location,intake_type,intake_condition,animal_type,intake_gender,intake_age,intake_breed,intake_color,animal_stay,outcome_time,date_of_birth,outcome_type,outcome_animal_type,outcome_gender,outcome_age,outcome_breed,outcome_color
0,2014-03-07 02:26:00,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White,A006100-1,2014-03-08 05:10:00,2007-07-09,Return to Owner,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
1,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,A006100-2,2014-12-20 04:35:00,2007-07-09,Return to Owner,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
2,2017-12-07 02:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,A006100-3,2017-12-07 12:00:00,2007-07-09,Return to Owner,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
3,2014-04-02 03:55:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,10 years,Dachshund,Tricolor,A047759-1,2014-04-07 03:12:00,2004-04-02,Transfer,Dog,Neutered Male,10 years,Dachshund,Tricolor
4,2013-11-16 09:02:00,12034 Research Blvd in Austin (TX),Public Assist,Injured,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White,A134067-1,2013-11-16 11:54:00,1997-10-16,Return to Owner,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White


*There were couple columns that were the same category, going to investigate these. Also converting age columns to age in months.*

In [25]:
# compare intake/outcome animal_type, gender, breed, and color
print(f'Number of animal type changes: {intakes_outcomes[intakes_outcomes['animal_type'] != intakes_outcomes['outcome_animal_type']].shape[0]}')
print(f'Number of neuters/spays: {intakes_outcomes[intakes_outcomes['intake_gender'] != intakes_outcomes['outcome_gender']].shape[0]}')
print(f'Number of breed changes: {intakes_outcomes[intakes_outcomes['intake_breed'] != intakes_outcomes['outcome_breed']].shape[0]}')
print(f'Number of color changes: {intakes_outcomes[intakes_outcomes['intake_color'] != intakes_outcomes['outcome_color']].shape[0]}')

Number of animal type changes: 0
Number of neuters/spays: 68026
Number of breed changes: 0
Number of color changes: 0


*No changes in animal type, breed, or color from intake to outcome, dropping the duplicate column.*

In [26]:
# dropping duplicate columns
intakes_outcomes.drop(columns=['outcome_animal_type', 'outcome_breed', 'outcome_color'], inplace=True)

# renaming columns without intake specifier
intakes_outcomes.rename(columns={'intake_breed': 'breed', 'intake_color': 'color'}, inplace=True)

In [27]:
# checking remaining null values
intakes_outcomes.isnull().sum()

intake_time          0
found_location       0
intake_type          0
intake_condition     0
animal_type          0
intake_gender        2
intake_age           1
breed                0
color                0
animal_stay          0
outcome_time         0
date_of_birth        0
outcome_type        46
outcome_gender       2
outcome_age         16
dtype: int64

*At most there are 67 observations with null values, dropping these rows.*

In [28]:
print(intakes_outcomes.shape)
intakes_outcomes.dropna(inplace=True)
print(intakes_outcomes.shape)

(167017, 15)
(166955, 15)


In [29]:
# function for converting age columns to age in months
def convert_age(age): 
    # add doc string for fucntion!!!!!!!!
    value, unit = age.split()
    value = abs(int(value)) # assume the nagetive age is typo 
    
    if 'year' in unit:
        return value * 12
    elif 'month' in unit:
        return value
    elif 'week' in unit:
        return round(float(value * 0.23), 2)
    elif 'day' in unit:
        return round(float(value * 0.033), 2)
    else:
        return 0 

In [30]:
intakes_outcomes['intake_age'] = intakes_outcomes['intake_age'].map(convert_age)
intakes_outcomes['outcome_age'] = intakes_outcomes['outcome_age'].map(convert_age)

In [31]:
# saving combined data to use in other notebooks
intakes_outcomes.to_csv('../data/combined-shelter-date.csv', index=False)

---
### Data Dictionary

All intake data is from [Austin Animal Center Intakes](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm/about_data) and outcome data is from [Austin Animal Center Outcomes](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238/about_data).

|feature|type|description|
|---|---|---|
|**intake_time**|*datetime*|Day and time animal is taken in by shelter|
|**found_location**|*str*|Where animal is found|
|**intake_type**|*str*|How animal is taken in|
|**intake_condition**|*str*|Animal's health condition upon intake|
|**animal_type**|*str*|Type of animal|
|**intake_gender**|*str*|Neuter/spay status upon intake|
|**intake_age**|*float*|Animal age in months upon intake|
|**breed**|*str*|Animal breed|
|**color**|*str*|Animal color|
|**animal_stay**|*str*|Animal identification based on stay in shelter|
|**outcome_time**|*datetime*|Day and time of animal outcome|
|**date_of_birth**|*datetime*|Animal's date of birth|
|**outcome_type**|*str*|Outcome of animal|
|**outcome_gender**|*str*|Neuter/spay status at outcome|
|**outcome_age**|*float*|Animal age at outcome|